# **Modelo Predictivo de Aprobación de Préstamos**
## *Clasificación Supervisada con Pipelines para Evaluación de Riesgo Crediticio*

---

### **Introducción**

La evaluación del riesgo crediticio es fundamental para las instituciones financieras. Determinar si un solicitante de préstamo cumplirá con sus obligaciones de pago es una decisión que involucra múltiples factores: perfil demográfico, historial crediticio, capacidad de pago y propósito del préstamo.

Un modelo predictivo de clasificación puede ayudar a:
- **Automatizar** la evaluación inicial de solicitudes
- **Reducir** el riesgo de default
- **Identificar** los factores más relevantes en la decisión
- **Optimizar** la cartera de préstamos

### **Objetivo del Estudio**

Desarrollar un modelo de **clasificación binaria** que prediga si un préstamo será aprobado/pagado exitosamente (1) o rechazado/default (0), utilizando:

1. **Pipelines de scikit-learn** para preprocesamiento y modelado
2. **Múltiples algoritmos** de clasificación
3. **Métricas de evaluación**: Precision, Recall, Accuracy, F1-Score, AUC-ROC

### **Los Datos**

El dataset contiene **45,000 solicitudes de préstamo** con las siguientes variables:

| Variable | Descripción |
|----------|-------------|
| `person_age` | Edad del solicitante |
| `person_gender` | Género |
| `person_education` | Nivel educativo |
| `person_income` | Ingresos anuales |
| `person_emp_exp` | Años de experiencia laboral |
| `person_home_ownership` | Tipo de vivienda |
| `loan_amnt` | Monto del préstamo solicitado |
| `loan_intent` | Propósito del préstamo |
| `loan_int_rate` | Tasa de interés |
| `loan_percent_income` | % del ingreso que representa el préstamo |
| `cb_person_cred_hist_length` | Años de historial crediticio |
| `credit_score` | Puntaje crediticio |
| `previous_loan_defaults_on_file` | Defaults previos (Sí/No) |
| **`loan_status`** | **Variable objetivo** (0=Default, 1=Pagado) |

---
## **1. Configuración del Entorno**

In [ ]:
# Importación de librerías
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

# Machine Learning y Pipelines
from sklearn.model_selection import train_test_split, cross_val_score, StratifiedKFold
from sklearn.preprocessing import StandardScaler, LabelEncoder, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer

# Modelos de clasificación
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier

# Métricas
from sklearn.metrics import (accuracy_score, precision_score, recall_score, 
                             f1_score, classification_report, confusion_matrix,
                             roc_auc_score, roc_curve, precision_recall_curve)

# Configuración de visualización
plt.style.use('seaborn-v0_8-whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)
plt.rcParams['font.size'] = 10

print("✓ Librerías importadas correctamente")

---
## **2. Carga y Exploración de Datos**

In [ ]:
# Cargar datos desde GitHub
url = 'https://raw.githubusercontent.com/Oscarpeg/ExelDireccionProyectos/main/loan_data.csv'
df = pd.read_csv(url)

print(f"Dataset cargado: {df.shape[0]:,} filas × {df.shape[1]} columnas")
df.head(10)

In [ ]:
# Información general
print("="*60)
print("INFORMACIÓN GENERAL DEL DATASET")
print("="*60)
print(f"\nDimensiones: {df.shape[0]:,} registros × {df.shape[1]} variables")
print(f"\nValores faltantes: {df.isnull().sum().sum()}")
print(f"\nTipos de datos:")
print(df.dtypes)

In [ ]:
# Estadísticas descriptivas
print("="*60)
print("ESTADÍSTICAS DESCRIPTIVAS")
print("="*60)
df.describe().round(2)

In [ ]:
# Distribución de la variable objetivo
print("="*60)
print("DISTRIBUCIÓN DE LA VARIABLE OBJETIVO")
print("="*60)

fig, axes = plt.subplots(1, 2, figsize=(12, 4))

# Conteo
counts = df['loan_status'].value_counts()
labels = ['Rechazado/Default (0)', 'Aprobado/Pagado (1)']
colors = ['#e74c3c', '#2ecc71']

axes[0].bar(labels, counts.values, color=colors)
axes[0].set_title('Distribución de loan_status', fontweight='bold')
axes[0].set_ylabel('Frecuencia')
for i, v in enumerate(counts.values):
    axes[0].text(i, v + 500, f'{v:,}', ha='center', fontweight='bold')

# Porcentaje
axes[1].pie(counts.values, labels=labels, autopct='%1.1f%%', colors=colors, explode=[0.02, 0.02])
axes[1].set_title('Proporción de Clases', fontweight='bold')

plt.tight_layout()
plt.show()

print(f"\nClase 0 (Rechazado/Default): {counts[0]:,} ({counts[0]/len(df)*100:.1f}%)")
print(f"Clase 1 (Aprobado/Pagado): {counts[1]:,} ({counts[1]/len(df)*100:.1f}%)")
print(f"\n⚠ NOTA: Dataset DESBALANCEADO - La clase minoritaria es solo {counts[1]/len(df)*100:.1f}%")

---
## **3. Análisis Exploratorio de Variables**

In [ ]:
# Análisis de variables numéricas vs loan_status
num_vars = ['person_age', 'person_income', 'loan_amnt', 'loan_int_rate', 
            'loan_percent_income', 'credit_score']

fig, axes = plt.subplots(2, 3, figsize=(15, 10))
axes = axes.flatten()

for i, var in enumerate(num_vars):
    df.boxplot(column=var, by='loan_status', ax=axes[i])
    axes[i].set_title(f'{var} por loan_status', fontweight='bold')
    axes[i].set_xlabel('loan_status')
    
plt.suptitle('Variables Numéricas por Estado del Préstamo', fontsize=14, fontweight='bold', y=1.02)
plt.tight_layout()
plt.show()

In [ ]:
# Tasa de aprobación por variables categóricas
cat_vars = ['person_gender', 'person_education', 'person_home_ownership', 
            'loan_intent', 'previous_loan_defaults_on_file']

fig, axes = plt.subplots(2, 3, figsize=(16, 10))
axes = axes.flatten()

for i, var in enumerate(cat_vars):
    tasa = df.groupby(var)['loan_status'].mean() * 100
    tasa.sort_values().plot(kind='barh', ax=axes[i], color='steelblue')
    axes[i].set_title(f'Tasa de Aprobación por {var}', fontweight='bold')
    axes[i].set_xlabel('% Aprobados')
    axes[i].axvline(x=22.2, color='red', linestyle='--', label='Promedio general')

axes[5].axis('off')  # Ocultar el 6to subplot vacío
plt.tight_layout()
plt.show()

In [ ]:
# HALLAZGO CRÍTICO: Defaults previos
print("="*60)
print("HALLAZGO CRÍTICO: DEFAULTS PREVIOS")
print("="*60)

for default in df['previous_loan_defaults_on_file'].unique():
    subset = df[df['previous_loan_defaults_on_file'] == default]
    tasa = subset['loan_status'].mean() * 100
    print(f"\nDefaults previos = {default}:")
    print(f"   Total: {len(subset):,} solicitudes")
    print(f"   Tasa de aprobación: {tasa:.1f}%")

print("\n" + "-"*60)
print("⚠ CONCLUSIÓN: Si el solicitante tiene defaults previos,")
print("  la probabilidad de aprobación es 0%. Este es el factor")
print("  más determinante en la decisión de crédito.")

In [ ]:
# Matriz de correlación
num_cols = ['person_age', 'person_income', 'person_emp_exp', 'loan_amnt',
            'loan_int_rate', 'loan_percent_income', 'cb_person_cred_hist_length',
            'credit_score', 'loan_status']

fig, ax = plt.subplots(figsize=(10, 8))
corr_matrix = df[num_cols].corr()
sns.heatmap(corr_matrix, annot=True, cmap='RdYlBu_r', center=0, fmt='.2f', ax=ax)
ax.set_title('Matriz de Correlación', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

print("\nCorrelaciones con loan_status:")
corr_target = corr_matrix['loan_status'].drop('loan_status').sort_values(key=abs, ascending=False)
for var, corr in corr_target.items():
    print(f"   {var:30}: {corr:+.4f}")

---
## **4. Preprocesamiento con Pipelines**

In [ ]:
# Definir features y target
features_num = ['person_age', 'person_income', 'person_emp_exp', 'loan_amnt',
                'loan_int_rate', 'loan_percent_income', 'cb_person_cred_hist_length',
                'credit_score']

features_cat = ['person_gender', 'person_education', 'person_home_ownership',
                'loan_intent', 'previous_loan_defaults_on_file']

X = df[features_num + features_cat]
y = df['loan_status']

print(f"Features numéricas: {len(features_num)}")
print(f"Features categóricas: {len(features_cat)}")
print(f"Total features: {len(features_num) + len(features_cat)}")
print(f"Muestras: {len(X):,}")

In [ ]:
# División de datos (estratificada para mantener proporción de clases)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

print("="*60)
print("DIVISIÓN DE DATOS")
print("="*60)
print(f"\nEntrenamiento: {len(X_train):,} muestras (80%)")
print(f"Prueba: {len(X_test):,} muestras (20%)")
print(f"\nDistribución en entrenamiento:")
print(f"   Clase 0: {(y_train == 0).sum():,} ({(y_train == 0).mean()*100:.1f}%)")
print(f"   Clase 1: {(y_train == 1).sum():,} ({(y_train == 1).mean()*100:.1f}%)")
print(f"\nDistribución en prueba:")
print(f"   Clase 0: {(y_test == 0).sum():,} ({(y_test == 0).mean()*100:.1f}%)")
print(f"   Clase 1: {(y_test == 1).sum():,} ({(y_test == 1).mean()*100:.1f}%)")

In [ ]:
# Crear Pipeline de preprocesamiento
print("="*60)
print("PIPELINE DE PREPROCESAMIENTO")
print("="*60)

# Preprocesador para variables numéricas
preprocessor_num = Pipeline([
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])

# Preprocesador para variables categóricas
preprocessor_cat = Pipeline([
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore', sparse_output=False))
])

# Combinar preprocesadores
preprocessor = ColumnTransformer(
    transformers=[
        ('num', preprocessor_num, features_num),
        ('cat', preprocessor_cat, features_cat)
    ])

print("""
Estructura del Pipeline:

┌─────────────────────────────────────────────────────────────┐
│                    PIPELINE COMPLETO                        │
├─────────────────────────────────────────────────────────────┤
│                                                             │
│  ┌─────────────────────┐   ┌─────────────────────┐         │
│  │ Variables Numéricas │   │ Variables Categóricas│        │
│  │  (8 features)       │   │  (5 features)        │        │
│  └──────────┬──────────┘   └──────────┬──────────┘         │
│             │                         │                     │
│             ▼                         ▼                     │
│  ┌─────────────────────┐   ┌─────────────────────┐         │
│  │ SimpleImputer       │   │ SimpleImputer       │         │
│  │ (strategy='median') │   │ (strategy='mode')   │         │
│  └──────────┬──────────┘   └──────────┬──────────┘         │
│             │                         │                     │
│             ▼                         ▼                     │
│  ┌─────────────────────┐   ┌─────────────────────┐         │
│  │ StandardScaler      │   │ OneHotEncoder       │         │
│  │ (μ=0, σ=1)         │   │                     │         │
│  └──────────┬──────────┘   └──────────┬──────────┘         │
│             │                         │                     │
│             └───────────┬─────────────┘                     │
│                         │                                   │
│                         ▼                                   │
│              ┌─────────────────────┐                        │
│              │    CLASIFICADOR     │                        │
│              │  (Random Forest,    │                        │
│              │   Logistic Reg, etc)│                        │
│              └─────────────────────┘                        │
│                                                             │
└─────────────────────────────────────────────────────────────┘
""")

---
## **5. Entrenamiento y Evaluación de Modelos**

In [ ]:
# Definir modelos
modelos = {
    'Logistic Regression': LogisticRegression(max_iter=1000, random_state=42),
    'Random Forest': RandomForestClassifier(n_estimators=100, random_state=42, n_jobs=-1),
    'Gradient Boosting': GradientBoostingClassifier(n_estimators=100, random_state=42),
    'KNN': KNeighborsClassifier(n_neighbors=5)
}

# Entrenar y evaluar
resultados = []
pipelines_entrenados = {}

print("="*70)
print("ENTRENAMIENTO Y EVALUACIÓN DE MODELOS")
print("="*70)

for nombre, modelo in modelos.items():
    print(f"\nEntrenando {nombre}...")
    
    # Crear pipeline completo
    pipeline = Pipeline([
        ('preprocessor', preprocessor),
        ('classifier', modelo)
    ])
    
    # Entrenar
    pipeline.fit(X_train, y_train)
    pipelines_entrenados[nombre] = pipeline
    
    # Predecir
    y_pred = pipeline.predict(X_test)
    
    # Probabilidades (si están disponibles)
    if hasattr(modelo, 'predict_proba'):
        y_proba = pipeline.predict_proba(X_test)[:, 1]
        auc = roc_auc_score(y_test, y_proba)
    else:
        auc = 0
    
    # Calcular métricas
    acc = accuracy_score(y_test, y_pred)
    prec = precision_score(y_test, y_pred)
    rec = recall_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)
    
    resultados.append({
        'Modelo': nombre,
        'Accuracy': acc,
        'Precision': prec,
        'Recall': rec,
        'F1-Score': f1,
        'AUC-ROC': auc
    })
    
    print(f"   Accuracy:  {acc:.4f}")
    print(f"   Precision: {prec:.4f}")
    print(f"   Recall:    {rec:.4f}")
    print(f"   F1-Score:  {f1:.4f}")
    print(f"   AUC-ROC:   {auc:.4f}")

# Tabla de resultados
df_resultados = pd.DataFrame(resultados).sort_values('F1-Score', ascending=False)
print("\n" + "="*70)
print("RESUMEN DE RESULTADOS")
print("="*70)
print(df_resultados.to_string(index=False))

In [ ]:
# Visualización comparativa
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Gráfico de barras
metricas = ['Accuracy', 'Precision', 'Recall', 'F1-Score', 'AUC-ROC']
x = np.arange(len(metricas))
width = 0.2

for i, (_, row) in enumerate(df_resultados.iterrows()):
    valores = [row['Accuracy'], row['Precision'], row['Recall'], row['F1-Score'], row['AUC-ROC']]
    axes[0].bar(x + i*width, valores, width, label=row['Modelo'])

axes[0].set_xlabel('Métrica')
axes[0].set_ylabel('Valor')
axes[0].set_title('Comparación de Métricas por Modelo', fontweight='bold')
axes[0].set_xticks(x + width * 1.5)
axes[0].set_xticklabels(metricas, rotation=45)
axes[0].legend(loc='lower right')
axes[0].set_ylim(0, 1)

# Curvas ROC
for nombre, pipeline in pipelines_entrenados.items():
    if hasattr(pipeline.named_steps['classifier'], 'predict_proba'):
        y_proba = pipeline.predict_proba(X_test)[:, 1]
        fpr, tpr, _ = roc_curve(y_test, y_proba)
        auc = roc_auc_score(y_test, y_proba)
        axes[1].plot(fpr, tpr, label=f'{nombre} (AUC={auc:.3f})')

axes[1].plot([0, 1], [0, 1], 'k--', label='Aleatorio')
axes[1].set_xlabel('Tasa de Falsos Positivos (FPR)')
axes[1].set_ylabel('Tasa de Verdaderos Positivos (TPR)')
axes[1].set_title('Curvas ROC', fontweight='bold')
axes[1].legend(loc='lower right')

plt.tight_layout()
plt.show()

---
## **6. Análisis Detallado del Mejor Modelo**

In [ ]:
# Seleccionar mejor modelo
mejor_nombre = df_resultados.iloc[0]['Modelo']
mejor_pipeline = pipelines_entrenados[mejor_nombre]

print(f"MEJOR MODELO: {mejor_nombre}")
print("="*60)

# Predicciones
y_pred = mejor_pipeline.predict(X_test)
y_proba = mejor_pipeline.predict_proba(X_test)[:, 1]

# Reporte de clasificación
print("\nREPORTE DE CLASIFICACIÓN")
print("-"*60)
print(classification_report(y_test, y_pred, target_names=['Rechazado (0)', 'Aprobado (1)']))

In [ ]:
# Matriz de confusión
cm = confusion_matrix(y_test, y_pred)

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Matriz absoluta
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
            xticklabels=['Rechazado', 'Aprobado'], 
            yticklabels=['Rechazado', 'Aprobado'], ax=axes[0])
axes[0].set_title(f'Matriz de Confusión - {mejor_nombre}', fontweight='bold')
axes[0].set_xlabel('Predicción')
axes[0].set_ylabel('Valor Real')

# Matriz normalizada
cm_norm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
sns.heatmap(cm_norm, annot=True, fmt='.2%', cmap='Blues',
            xticklabels=['Rechazado', 'Aprobado'], 
            yticklabels=['Rechazado', 'Aprobado'], ax=axes[1])
axes[1].set_title('Matriz de Confusión Normalizada', fontweight='bold')
axes[1].set_xlabel('Predicción')
axes[1].set_ylabel('Valor Real')

plt.tight_layout()
plt.show()

# Interpretación
tn, fp, fn, tp = cm.ravel()
print("\nINTERPRETACIÓN DE LA MATRIZ DE CONFUSIÓN")
print("="*60)
print(f"\nVerdaderos Negativos (TN): {tn:,}")
print(f"   → Préstamos correctamente rechazados (habrían hecho default)")
print(f"\nFalsos Positivos (FP): {fp:,}")
print(f"   → Préstamos aprobados incorrectamente (PÉRDIDA PARA EL BANCO)")
print(f"\nFalsos Negativos (FN): {fn:,}")
print(f"   → Préstamos rechazados incorrectamente (OPORTUNIDAD PERDIDA)")
print(f"\nVerdaderos Positivos (TP): {tp:,}")
print(f"   → Préstamos correctamente aprobados (clientes que pagan)")

In [ ]:
# Explicación de métricas
print("="*70)
print("EXPLICACIÓN DE MÉTRICAS")
print("="*70)
print("""
┌─────────────────────────────────────────────────────────────────────┐
│ ACCURACY (Exactitud)                                                │
│ ─────────────────────                                               │
│ Fórmula: (TP + TN) / Total                                         │
│ Interpretación: % de predicciones correctas                         │
│                                                                     │
│ ⚠ CUIDADO: En datasets desbalanceados puede ser engañosa.          │
│   Un modelo que siempre prediga "Rechazado" tendría 77.8% accuracy │
├─────────────────────────────────────────────────────────────────────┤
│ PRECISION                                                           │
│ ─────────                                                           │
│ Fórmula: TP / (TP + FP)                                            │
│ Pregunta: De los préstamos que APROBAMOS, ¿cuántos pagan?          │
│                                                                     │
│ Alta Precision = Pocos préstamos malos aprobados                   │
│ Importante cuando: El costo de aprobar un mal préstamo es alto     │
├─────────────────────────────────────────────────────────────────────┤
│ RECALL (Sensibilidad)                                               │
│ ─────────────────────                                               │
│ Fórmula: TP / (TP + FN)                                            │
│ Pregunta: De los buenos clientes, ¿a cuántos aprobamos?            │
│                                                                     │
│ Alto Recall = Pocos buenos clientes rechazados                     │
│ Importante cuando: Perder buenos clientes tiene alto costo         │
├─────────────────────────────────────────────────────────────────────┤
│ F1-SCORE                                                            │
│ ────────                                                            │
│ Fórmula: 2 × (Precision × Recall) / (Precision + Recall)           │
│ Interpretación: Balance entre Precision y Recall                   │
│                                                                     │
│ Útil cuando: Las clases están desbalanceadas                       │
├─────────────────────────────────────────────────────────────────────┤
│ AUC-ROC                                                             │
│ ───────                                                             │
│ Área bajo la curva ROC (0.5 = aleatorio, 1.0 = perfecto)           │
│ Mide la capacidad de distinguir entre clases                       │
│                                                                     │
│ AUC > 0.9 = Excelente discriminación                               │
│ AUC 0.8-0.9 = Buena discriminación                                 │
│ AUC 0.7-0.8 = Aceptable                                            │
└─────────────────────────────────────────────────────────────────────┘
""")

# Métricas del mejor modelo
print(f"\nMÉTRICAS DEL MODELO {mejor_nombre.upper()}:")
print(f"   Accuracy:  {accuracy_score(y_test, y_pred):.4f}")
print(f"   Precision: {precision_score(y_test, y_pred):.4f}")
print(f"   Recall:    {recall_score(y_test, y_pred):.4f}")
print(f"   F1-Score:  {f1_score(y_test, y_pred):.4f}")
print(f"   AUC-ROC:   {roc_auc_score(y_test, y_proba):.4f}")

---
## **7. Importancia de Variables**

In [ ]:
# Obtener importancia de variables
if mejor_nombre in ['Random Forest', 'Gradient Boosting']:
    importancia = mejor_pipeline.named_steps['classifier'].feature_importances_
    
    # Obtener nombres de features
    cat_encoder = mejor_pipeline.named_steps['preprocessor'].named_transformers_['cat']
    cat_feature_names = cat_encoder.named_steps['onehot'].get_feature_names_out(features_cat)
    all_features = features_num + list(cat_feature_names)
    
    # Crear DataFrame
    imp_df = pd.DataFrame({
        'Variable': all_features,
        'Importancia': importancia
    }).sort_values('Importancia', ascending=True)
    
    # Visualizar
    fig, ax = plt.subplots(figsize=(10, 10))
    colors = plt.cm.RdYlGn(np.linspace(0.2, 0.8, len(imp_df)))
    ax.barh(imp_df['Variable'], imp_df['Importancia'], color=colors)
    ax.set_xlabel('Importancia')
    ax.set_title(f'Importancia de Variables - {mejor_nombre}', fontsize=14, fontweight='bold')
    plt.tight_layout()
    plt.show()
    
    print("\nTOP 10 VARIABLES MÁS IMPORTANTES:")
    print("="*60)
    for i, row in imp_df.tail(10).iloc[::-1].iterrows():
        print(f"   {row['Variable']:40}: {row['Importancia']:.4f}")

---
## **8. Validación Cruzada**

In [ ]:
# Validación cruzada estratificada
print("="*60)
print("VALIDACIÓN CRUZADA ESTRATIFICADA (5-Fold)")
print("="*60)
print("\nLa validación cruzada evalúa el modelo en múltiples particiones")
print("de los datos para obtener una estimación más robusta del rendimiento.\n")

cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

cv_results = []
for nombre, modelo in modelos.items():
    pipeline = Pipeline([
        ('preprocessor', preprocessor),
        ('classifier', modelo)
    ])
    
    # Calcular scores para múltiples métricas
    acc_scores = cross_val_score(pipeline, X, y, cv=cv, scoring='accuracy')
    f1_scores = cross_val_score(pipeline, X, y, cv=cv, scoring='f1')
    
    cv_results.append({
        'Modelo': nombre,
        'Accuracy_mean': acc_scores.mean(),
        'Accuracy_std': acc_scores.std(),
        'F1_mean': f1_scores.mean(),
        'F1_std': f1_scores.std()
    })
    
    print(f"{nombre}:")
    print(f"   Accuracy: {acc_scores.mean():.4f} (+/- {acc_scores.std()*2:.4f})")
    print(f"   F1-Score: {f1_scores.mean():.4f} (+/- {f1_scores.std()*2:.4f})")
    print()

---
## **9. Predicción de Nuevos Casos**

In [ ]:
# Ejemplo de predicción para nuevos solicitantes
print("="*60)
print("PREDICCIÓN PARA NUEVOS SOLICITANTES")
print("="*60)

# Crear casos de ejemplo
nuevos_casos = pd.DataFrame({
    'person_age': [25, 35, 45],
    'person_gender': ['male', 'female', 'male'],
    'person_education': ['Bachelor', 'Master', 'High School'],
    'person_income': [50000, 85000, 40000],
    'person_emp_exp': [2, 8, 15],
    'person_home_ownership': ['RENT', 'MORTGAGE', 'OWN'],
    'loan_amnt': [10000, 25000, 8000],
    'loan_intent': ['EDUCATION', 'HOMEIMPROVEMENT', 'MEDICAL'],
    'loan_int_rate': [12.5, 9.0, 15.0],
    'loan_percent_income': [0.20, 0.29, 0.20],
    'cb_person_cred_hist_length': [3, 10, 20],
    'credit_score': [650, 720, 580],
    'previous_loan_defaults_on_file': ['No', 'No', 'Yes']
})

print("\nCasos a evaluar:")
print(nuevos_casos[['person_age', 'person_income', 'loan_amnt', 'credit_score', 'previous_loan_defaults_on_file']].to_string())

# Predecir
predicciones = mejor_pipeline.predict(nuevos_casos)
probabilidades = mejor_pipeline.predict_proba(nuevos_casos)

print("\n" + "-"*60)
print("RESULTADOS:")
print("-"*60)

for i in range(len(nuevos_casos)):
    resultado = "APROBADO" if predicciones[i] == 1 else "RECHAZADO"
    prob = probabilidades[i][1] * 100
    print(f"\nSolicitante {i+1}:")
    print(f"   Edad: {nuevos_casos.iloc[i]['person_age']}, Ingresos: ${nuevos_casos.iloc[i]['person_income']:,}")
    print(f"   Monto solicitado: ${nuevos_casos.iloc[i]['loan_amnt']:,}")
    print(f"   Credit Score: {nuevos_casos.iloc[i]['credit_score']}")
    print(f"   Defaults previos: {nuevos_casos.iloc[i]['previous_loan_defaults_on_file']}")
    print(f"   → Decisión: {resultado}")
    print(f"   → Probabilidad de pago: {prob:.1f}%")

---
## **10. Conclusiones**

In [ ]:
print("="*70)
print("CONCLUSIONES DEL ANÁLISIS")
print("="*70)

print(f"""
1. CARACTERIZACIÓN DEL DATASET
   ────────────────────────────
   • 45,000 solicitudes de préstamo analizadas
   • Dataset DESBALANCEADO: 77.8% rechazados, 22.2% aprobados
   • 13 variables predictoras (8 numéricas, 5 categóricas)

2. HALLAZGO CRÍTICO: DEFAULTS PREVIOS
   ──────────────────────────────────
   • Si el solicitante tiene defaults previos → 0% de aprobación
   • Sin defaults previos → 45.2% de aprobación
   • Esta es la variable más determinante en la decisión

3. RENDIMIENTO DEL MEJOR MODELO ({mejor_nombre})
   ──────────────────────────────────────────────
   • Accuracy:  {accuracy_score(y_test, y_pred):.4f} ({accuracy_score(y_test, y_pred)*100:.1f}% de predicciones correctas)
   • Precision: {precision_score(y_test, y_pred):.4f} (De los aprobados, {precision_score(y_test, y_pred)*100:.1f}% pagan)
   • Recall:    {recall_score(y_test, y_pred):.4f} (Detectamos {recall_score(y_test, y_pred)*100:.1f}% de buenos clientes)
   • F1-Score:  {f1_score(y_test, y_pred):.4f} (Balance precision-recall)
   • AUC-ROC:   {roc_auc_score(y_test, y_proba):.4f} (Excelente capacidad discriminativa)

4. VARIABLES MÁS IMPORTANTES
   ─────────────────────────
   • loan_int_rate (Tasa de interés)
   • loan_percent_income (% del ingreso)
   • previous_loan_defaults_on_file (Defaults previos)
   • person_income (Ingresos)

5. USO DE PIPELINES
   ─────────────────
   • Los Pipelines garantizan preprocesamiento consistente
   • Evitan data leakage entre entrenamiento y prueba
   • Facilitan la reproducibilidad y el despliegue

6. MÉTRICAS DE EVALUACIÓN
   ──────────────────────
   • PRECISION: Importante para minimizar pérdidas (préstamos malos aprobados)
   • RECALL: Importante para no perder buenos clientes
   • En banca, suele priorizarse PRECISION para evitar defaults

7. RECOMENDACIONES
   ────────────────
   • Rechazar automáticamente solicitantes con defaults previos
   • Evaluar cuidadosamente préstamos con alto % del ingreso
   • Considerar ajustar el umbral de decisión según apetito de riesgo
""")

print("="*70)
print("FIN DEL ANÁLISIS")
print("="*70)